# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/'My Drive/Transsight/QSR'
%ls

Mounted at /content/drive
/content/drive/My Drive/Transsight/QSR
Churn_prediction.ipynb  docs/                            QRS_EDA.ipynb
data/                   Product_Affinity_analysis.ipynb  RFM.ipynb


# Libraries

In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Product Affinity analysis

## Product data

### Read data

In [10]:
product_df = pd.read_csv("data/ProductData -masked.csv")
product_df['Date'] = pd.to_datetime(product_df["Actual Date"])
product_df.rename(columns={'CHECK_ID': 'Transaction_ID', 'recordID': 'Customer_ID', 'menuItemPOSRef': 'Product_ID'}, inplace=True)
product_df['Quantity'] = 1
print("Number of rows : ", len(product_df))
product_df.head(2)

Number of rows :  59154


,CONTACT_NUMBER,Transaction_ID,city,ONLINE_ORDER_TYPE,Customer_ID,Product_ID,Actual Date,Date,Quantity
0,9870XXXX53606,711863273,HYDERABAD/SECUNDERABAD,0171-6208991 WebMobile,84258674,310128,14 May 18,2018-05-14,1
1,9870XXXX51763,716686990,NaN,NaN,84258093,310128,27 May 18,2018-05-27,1


### Basic Info

##### Null values

In [12]:
count=product_df.isnull().sum().sort_values(ascending=False)
ratio=count/len(product_df)
pd.concat([count,ratio],axis=1,keys=['count','ratio'])

,count,ratio
city,23479,0.396913
ONLINE_ORDER_TYPE,20774,0.351185
CONTACT_NUMBER,0,0.000000
Transaction_ID,0,0.000000
Customer_ID,0,0.000000
Product_ID,0,0.000000
Actual Date,0,0.000000
Date,0,0.000000
Quantity,0,0.000000


#### Unique values

In [11]:
product_df.nunique()

CONTACT_NUMBER       59154
Transaction_ID       59154
city                   721
ONLINE_ORDER_TYPE    38380
Customer_ID            917
Product_ID               5
Actual Date             52
Date                    52
Quantity                 1
dtype: int64

### Prepare basket data

In [16]:
basket_data = (product_df.groupby(['Transaction_ID', 'Product_ID'])['Quantity']
            .sum().unstack().reset_index().fillna(0)
            .set_index('Transaction_ID'))

basket_data.head()

Product_ID,120143,310128,310130,401006,401007
Transaction_ID,,,,,
697365617,0.0,0.0,0.0,0.0,1.0
697367101,0.0,0.0,0.0,1.0,0.0
697367377,0.0,1.0,0.0,0.0,0.0
697367538,0.0,0.0,0.0,0.0,1.0
697367577,0.0,1.0,0.0,0.0,0.0


### Calculate Support for Products

In [20]:
support_data = apriori(basket_data, min_support=0.05, use_colnames=True)
support_data

,support,itemsets
0,0.127650,(120143)
1,0.558576,(310128)
2,0.081888,(401006)
3,0.211820,(401007)


#### Calculate Rules based on Product's Support

In [26]:
rules_data = association_rules(support_data, metric="lift", min_threshold=0.01)
rules_data

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


Data is too limited that no rules can be generated. One product is bought only once by a customer at a time.